In [1]:
# --- Set location of XASNet code
import sys
sys.path.append('/home/samjhall/github/XASNet-XAI/src')
#sys.path.append('D:\github\XASNet-XAI\src')
# --- Standard libraries
import os.path as osp
import numpy as np
import pickle as pkl
# --- PyTorch and PyG
import torch
from torch_geometric.loader import DataLoader
import torch_scatter
# --- XASNet
from XASNet.data import QM9_XAS
from XASNet.data import save_split
from XASNet.models import XASNet_GNN, XASNet_GAT, XASNet_GraphNet
from XASNet.trainer import GNNTrainer

In [2]:
# --- Load in the dataset
root = './XASNet-data/atom_dataset.pt'
go_spec = QM9_XAS(root=root,
                  raw_dir='./XASNet-data/',
                  spectra=[])

In [3]:
# --- Print details of the dataset
print(go_spec)
print('------------')
print(f'Number of graphs: {len(go_spec)}')
print(f'Number of features: {go_spec.num_features}')
print('')

# --- Print details of the first molecule/graph in dataset
data = go_spec[0]

print(data)
print('------------')
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Has isolated nodes: {data.has_isolated_nodes()}')
print(f'Has self loops: {data.has_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')

QM9_XAS(7946)
------------
Number of graphs: 7946
Number of features: 15

Data(x=[32, 15], edge_index=[2, 78], edge_attr=[78, 6], spectrum=[200], vector=[15], idx=[1], smiles='[c:0]12[c:4]3[c:8]4[c:10]5[c:11]([CH:25]=[O:29])[cH:13][c:14]6[c:12]4[c:17]4[c:19]([c:18]([CH:26]=[O:31])[c:15]6[OH:16])=[CH:20][CH:22]6[C:23]([c:21]14)([CH:24]=[CH:1][C:2]2=[CH:3][CH2:5][C:6]3([CH:27]=[O:30])[CH:7]=[CH:9]5)[O:28]6', atom_num=[1], neighbors=[3])
------------
Number of nodes: 32
Number of edges: 78
Average node degree: 2.44
Has isolated nodes: False
Has self loops: False
Is undirected: True


In [4]:
# --- Create spilt file with the dataset
# --- split into test, validation and test datasets
idxs = save_split(
    path='./raw/xasnet-atom-split.npz',
    ndata=len(go_spec),
    ntrain=5960,
    nval=1192,
    ntest=794,
    save_split=True,
    shuffle=True, 
    print_nsample=True
)

In [5]:
# --- Create variables for each dataset split
train_go = [go_spec[i] for i in idxs['train']]
val_go = [go_spec[i] for i in idxs['val']]
test_go = [go_spec[i] for i in idxs['test']]

# --- Save datasets splits into dataloaders
train_loader = DataLoader(train_go, batch_size=128, shuffle=True)
val_loader = DataLoader(val_go, batch_size=64, shuffle=True)
test_loader = DataLoader(test_go, batch_size=64, shuffle=False)

In [6]:
# --- Save the dataloader to a file
torch.save(test_go, './XASNet-data/test_atom_dataset.pt')

In [7]:
# --- Define cost functions
def RSE_loss(prediction, target):
    dE = (300 - 280) / 200
    nom = torch.sum(dE*torch.pow((target-prediction), 2))
    denom = torch.sum(dE*target)
    return torch.sqrt(nom) / denom 

def RMSE(prediction, target):
    return torch.sqrt(torch.mean((target - prediction)**2))

In [8]:
# --- Set name for ML model
model_name = 'xasnet_model'
# --- Set number of epochs to run
num_epochs = 300
# --- Set the learning rate 
lr = 0.01
# --- Milestones to reduce learning rate in steps 
milestones = np.arange(10, 100, 10).tolist()

##### XASNet_GNN

In [9]:
# --- Set device for model to run on
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# --- Create the type of ML model you want to run
xasnet_gnn = XASNet_GNN(
    gnn_name = 'gat', # model type
    in_channels = [15, 512, 512, 256, 128], # input nodes for each layer
    out_channels = [512, 512, 256, 128, 200], # output nodes for each layer
    num_targets = 200, # nodes for final output
    num_layers = 5, # number of total layers
    heads = 1
).to(device)

# --- Location to save model
path_to_model = osp.join('./best_model,', model_name)

# --- Check if there is an already existing model
if osp.exists(path_to_model):
    xasnet_gnn.load_state_dict(torch.load(path_to_model))
else:
    print('Model is not loaded.')

Model is not loaded.


In [10]:
# --- View the details of the created model
xasnet_gnn

XASNet_GNN(
  (interaction_layers): ModuleList(
    (0): GATConv(15, 512, heads=1)
    (1): ReLU(inplace=True)
    (2): GATConv(512, 512, heads=1)
    (3): ReLU(inplace=True)
    (4): GATConv(512, 256, heads=1)
    (5): ReLU(inplace=True)
    (6): GATConv(256, 128, heads=1)
    (7): ReLU(inplace=True)
    (8): GATConv(128, 200, heads=1)
  )
  (dropout): Dropout(p=0.8, inplace=False)
  (out): Linear(in_features=200, out_features=200, bias=True)
)

#### XASNet_GAT

In [11]:
# --- Set device for model to run on
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# --- Create the type of ML model you want to run
xasnet_gat = XASNet_GAT(
    gat_type = 'gatv2_custom', # model type
    node_features_dim = 15,
    in_channels = [512, 512, 256, 128], # input nodes for each layer
    out_channels = [512, 256, 128, 200], # output nodes for each layer
    targets = 200, # nodes for final output
    n_layers = 4, # number of total layers
    n_heads = 1,
    use_residuals = True,
    use_jk = True
).to(device)

# --- Location to save model
path_to_model = osp.join('./best_model,', model_name)

# --- Check if there is an already existing model
if osp.exists(path_to_model):
    xasnet_gnn.load_state_dict(torch.load(path_to_model))
else:
    print('Model is not loaded.')

Model is not loaded.


In [12]:
xasnet_gat

XASNet_GAT(
  (pre_layer): LinearLayer(
    (linear): Linear(in_features=15, out_features=512, bias=False)
    (_activation): ReLU(inplace=True)
  )
  (res_block): Residual_block(
    (res_layers): Sequential(
      (0): LinearLayer(
        (linear): Linear(in_features=512, out_features=512, bias=False)
        (_activation): ReLU(inplace=True)
      )
      (1): LinearLayer(
        (linear): Linear(in_features=512, out_features=512, bias=False)
        (_activation): ReLU(inplace=True)
      )
      (2): LinearLayer(
        (linear): Linear(in_features=512, out_features=512, bias=False)
        (_activation): ReLU(inplace=True)
      )
      (3): LinearLayer(
        (linear): Linear(in_features=512, out_features=512, bias=False)
        (_activation): ReLU(inplace=True)
      )
    )
  )
  (gat_layers): ModuleList(
    (0): GATv2LayerCus(512, 512)
    (1): ReLU(inplace=True)
    (2): GATv2LayerCus(512, 256)
    (3): ReLU(inplace=True)
    (4): GATv2LayerCus(256, 128)
    (5): ReLU

#### XASNet_GraphNet

In [13]:
# --- Set device for model to run on
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# --- Create the type of ML model you want to run
xasnet_graphnet = XASNet_GraphNet(
    node_dim = 15,
    edge_dim = 6,
    hidden_channels = 512,
    out_channels = 200, # output nodes for each layer
    gat_hidd = 512,
    gat_out = 100,
    n_targets = 200, # nodes for final output
    n_layers = 3 # number of total layers
).to(device)

# --- Location to save model
path_to_model = osp.join('./best_model,', model_name)

# --- Check if there is an already existing model
if osp.exists(path_to_model):
    xasnet_gnn.load_state_dict(torch.load(path_to_model))
else:
    print('Model is not loaded.')

Model is not loaded.


In [14]:
xasnet_graphnet

XASNet_GraphNet(
  (graphnets): ModuleList(
    (0): GraphNetwork(
      (gatencoder): GATEncoder(
        (gats): ModuleList(
          (0): GATv2Conv(15, 512, heads=3)
          (1): ReLU(inplace=True)
          (2): GATv2Conv(1536, 512, heads=3)
          (3): ReLU(inplace=True)
          (4): GATv2Conv(1536, 512, heads=3)
          (5): ReLU(inplace=True)
          (6): GATv2Conv(1536, 100, heads=1)
        )
      )
      (node_model): NodeModel(
        (mlp): Sequential(
          (0): Linear(in_features=127, out_features=512, bias=True)
          (1): ReLU(inplace=True)
          (2): Linear(in_features=512, out_features=200, bias=True)
          (3): ReLU(inplace=True)
          (4): LayerNorm((200,), eps=1e-05, elementwise_affine=True)
        )
      )
      (edge_model): EdgeModel(
        (mlp): Sequential(
          (0): Linear(in_features=506, out_features=512, bias=True)
          (1): ReLU(inplace=True)
          (2): Linear(in_features=512, out_features=200, bias=True

#### Train Model

In [11]:
chosen_model = xasnet_gnn

In [12]:
# --- Set additional ML parameters
optimizer = torch.optim.AdamW(chosen_model.parameters(), lr=lr)
loss_fn = torch.nn.L1Loss()
loss_fn2 = torch.nn.MSELoss()
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, 
                                                 milestones=milestones,
                                                 gamma=0.8)

In [13]:
# --- Create trainier
trainer = GNNTrainer(model = chosen_model,
                     model_name = model_name,
                     device = device,
                     metric_path = './metrics')

In [14]:
# --- Train the ML model
trainer.train_val(train_loader, val_loader, optimizer, RMSE,
                  scheduler, num_epochs, write_every=25, train_graphnet=False)

  0%|          | 0/300 [00:00<?, ?it/s]/home/samjhall/anaconda3/envs/xasnet-xai/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:432: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "
  0%|          | 1/300 [00:00<04:11,  1.19it/s]

time = 0.01 mins mins
epoch 0 | average train loss = 0.00177  and average validation loss = 0.00327  |learning rate = 0.01000


  9%|▊         | 26/300 [00:15<02:41,  1.69it/s]

time = 0.26 mins mins
epoch 25 | average train loss = 0.00161  and average validation loss = 0.00326  |learning rate = 0.00640


 17%|█▋        | 51/300 [00:30<02:34,  1.61it/s]

...

 84%|████████▎ | 251/300 [02:13<00:24,  1.98it/s]

time = 2.22 mins mins
epoch 250 | average train loss = 0.00161  and average validation loss = 0.00326  |learning rate = 0.00134


 92%|█████████▏| 276/300 [02:26<00:11,  2.06it/s]

time = 2.43 mins mins
epoch 275 | average train loss = 0.00161  and average validation loss = 0.00326  |learning rate = 0.00134


100%|██████████| 300/300 [02:37<00:00,  1.90it/s]
